# QuakeML to json


<div class="alert alert-warning">

**Warning**: Much of this functionality will eventually be moved to obspy, see [this PR](https://github.com/obspy/obspy/pull/2210).

</div>

The following demonstrates obsplus' ability to serialize obspy catalog objects into json. All such conversions should be lossless.

## Get a catalog

In [1]:
import obsplus

crandall = obsplus.load_dataset("crandall_test")

cat = crandall.event_client.get_events()


## json conversions

In [2]:
import obsplus

# convert to json str
json_str = obsplus.cat_to_json(cat)

# print sample

# convert back
cat2 = obsplus.json_to_cat(json_str)

/home/runner/micromamba/envs/test/lib/python3.12/site-packages/obspy/core/util/attribdict.py:119: UserWarning: Setting attribute "seed_string" which is not a default attribute ("network_code", "station_code", "channel_code", "location_code", "resource_uri").
  warnings.warn(msg)


In [3]:
# json serialization should be lossless after handling Quantity Errors
# this won't be needed once obspy 1.2.0 is released.
import obspy.core.event as ev

from obsplus.utils import yield_obj_parent_attr


def _remove_empty_quantity_errors(catalog):
    """
    Copy the catalog and set all empty QunatityErrors to None.
    This is needed to check equality of catalogs that may have
    None or empty QuantityErrors.

    Fixed in https://github.com/obspy/obspy/pull/2185
    """
    cat = catalog.copy()
    for obj, parent, attr in yield_obj_parent_attr(cat, cls=ev.QuantityError):
        if not obj:
            setattr(parent, attr, None)
    return cat


cat1 = _remove_empty_quantity_errors(cat)

cat2 = _remove_empty_quantity_errors(cat2)

assert cat1 == cat2